In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
import plotly.graph_objs as go

function kdistance() - used to find the distance of a point with its kth nearest neighbour

In [2]:
def kdistance(k,pt,df):
    distance_from_all_points = []  # Store the values of distances between all sets of points
    for y in df:
     distance_from_all_points.append(np.linalg.norm(pt-y))
    distance_from_all_points.sort()
    return distance_from_all_points[k+1] # return kth smallest distance

function reachabilityDistance() - used to find reachability distance between 2 points, which is, min{dist(pt1,pt2),kdistance(pt2)}

In [3]:
def reachabilityDistance(pt1,pt2,k,df,kdist_val):
    dist = np.linalg.norm(pt2-pt1) # Euclidean distance between pt1 and pt2
    dist_k = kdist_val[tuple(pt2)] # k distance of pt2
    if(dist > dist_k):
        return dist
    else:
        return dist_k

function nearestNeighbours() - returns the k-nearest neighbours of a given point

In [4]:
def nearestNeighbours(k,pt,df):
    distance_from_all_points = []  # Store the values of distances between all sets of points
    for i in range(0,len(df)):
     distance_from_all_points.append(np.linalg.norm(pt-df[i]))
    sorted_indices = np.argsort(distance_from_all_points)
    ans = df[sorted_indices] # stores indices of points sorted in order of distance from given point
    return ans[1:k+1] # return indices of k nearest neighbours

function lrd() - calculates the locl reachability density of a given point

In [5]:
def lrd(pt,k,df,nn,kdist_val):
    s = 0   # Stores the value of reachability distance of pt
    for y in nn[tuple(pt)]:
        s = s + reachabilityDistance(pt,y,k,df,kdist_val)
    s = s/k
    if(s == 0):
        return -1
    else:
        return 1/s

function lof() - returns the lof value(outlier score) of a given point

In [6]:
def lof(pt,k,df,nn,kdist_val):
    s = 0  # Stores the value of lof of the point pt
    t = lrd(pt,k,df,nn,kdist_val)
    for y in nn[tuple(pt)]:
        s = s + (lrd(y,k,df,nn,kdist_val)/t)
    s = s/k   
    return s

Reading from subset_selection_cause.xlsx

In [7]:
X = pd.read_excel("~/DMFinalTerm_TeamID_05/data/subset_selection_cause.xlsx")
X

Index       State/UT  Year  C1  C2  C3  Male  Female
0         0  A & N ISLANDS  2001   0   0   1    27      19
1         0  A & N ISLANDS  2001   0   1   0    14       9
2         0  A & N ISLANDS  2001   1   0   0    38      22
3         1  A & N ISLANDS  2002   0   0   1    33      16
4         1  A & N ISLANDS  2002   0   1   0    18      18
...     ...            ...   ...  ..  ..  ..   ...     ...
1255    418    WEST BENGAL  2011   0   1   0  3625    3259
1256    418    WEST BENGAL  2011   1   0   0  4856    2833
1257    419    WEST BENGAL  2012   0   0   1     0       0
1258    419    WEST BENGAL  2012   0   1   0     0       0
1259    419    WEST BENGAL  2012   1   0   0     0       0

[1260 rows x 8 columns]

In [8]:
Y = X.drop(['State/UT','Index','Year','C1','C2','C3'],axis=1)
Y

Male  Female
0       27      19
1       14       9
2       38      22
3       33      16
4       18      18
...    ...     ...
1255  3625    3259
1256  4856    2833
1257     0       0
1258     0       0
1259     0       0

[1260 rows x 2 columns]

In [9]:
cols = Y.columns[1:]

Converting the dataset to a numpy array

In [10]:
x=Y['Male']
y=Y['Female']
d=Y[['Male','Female']]
e=np.array([[d.iloc[0][0],d.iloc[0][1]]])
for i in range(1,len(d)):
    e=np.append(e,[[d.iloc[i][0],d.iloc[i][1]]],axis=0)

e

array([[27, 19],
       [14,  9],
       [38, 22],
       ...,
       [ 0,  0],
       [ 0,  0],
       [ 0,  0]], dtype=int64)

Pre-computing kdistance for all points

In [11]:
kdist_val = {} # stores the value of k distance of all points
for y in e:
    kdist_val[tuple(y)] = kdistance(25,y,e)
kdist_val    

{(27, 19): 7.280109889280518,
 (14, 9): 3.605551275463989,
 (38, 22): 8.246211251235321,
 (33, 16): 6.708203932499369,
 (18, 18): 8.246211251235321,
 (40, 19): 7.615773105863909,
 (19, 13): 5.385164807134504,
 (24, 17): 6.4031242374328485,
 (27, 13): 5.0,
 (32, 16): 6.082762530298219,
 (28, 16): 6.4031242374328485,
 (21, 9): 4.123105625617661,
 (30, 23): 9.899494936611665,
 (9, 2): 2.8284271247461903,
 (47, 28): 6.324555320336759,
 (43, 19): 7.211102550927978,
 (5, 2): 2.23606797749979,
 (48, 16): 8.54400374531753,
 (17, 7): 3.605551275463989,
 (50, 33): 7.280109889280518,
 (38, 18): 8.0,
 (16, 4): 3.605551275463989,
 (44, 23): 7.0710678118654755,
 (30, 15): 6.324555320336759,
 (3, 2): 2.0,
 (53, 28): 7.0710678118654755,
 (39, 13): 7.280109889280518,
 (14, 5): 3.0,
 (59, 26): 9.0,
 (30, 14): 6.082762530298219,
 (13, 6): 3.0,
 (51, 22): 7.0710678118654755,
 (38, 12): 8.06225774829855,
 (33, 27): 12.041594578792296,
 (2333, 1515): 347.476617918386,
 (2277, 1628): 427.9684567815717,
 (176

Pre-computing the k-nearest neighbours for all points

In [12]:
nn = {} # stores the k nearest neighbours of all points
for y in e:
    nn[tuple(y)] = nearestNeighbours(25,y,e)
nn[tuple(e[0])]    

array([[26, 17],
       [28, 16],
       [24, 17],
       [24, 17],
       [27, 15],
       [28, 15],
       [31, 22],
       [23, 16],
       [30, 15],
       [30, 23],
       [22, 20],
       [22, 16],
       [24, 14],
       [30, 14],
       [32, 16],
       [27, 13],
       [28, 13],
       [33, 17],
       [29, 13],
       [33, 16],
       [33, 22],
       [33, 16],
       [27, 12],
       [26, 12],
       [29, 12]], dtype=int64)

Computing the outlier score(lof value) for all points

In [13]:
outlier_score = [] # stores the outlier score(lof value) of all points
for y in e:
    outlier_score.append(lof(y,25,e,nn,kdist_val))

In [14]:
outlier_score

[1.114866324637646,
 1.0053139557164392,
 1.044284925941682,
 1.0425959814349186,
 1.3186126194098222,
 1.009129567047198,
 1.1026523010666067,
 1.1327008207213942,
 1.014033923032131,
 1.041046848091475,
 1.0227703837450322,
 1.0013389697016613,
 1.208042732427007,
 1.010613787249961,
 0.9750092596517647,
 0.9945588543721092,
 1.0321025081861555,
 1.0406571449436735,
 1.0425959814349186,
 0.9832082022992482,
 0.9945182509862944,
 1.0022353618965194,
 1.0155689745559628,
 0.9833130005514416,
 1.0241743101552963,
 1.1379182915087485,
 0.9730622082475229,
 1.0270490188508576,
 0.9829315929373714,
 1.0218753490680235,
 1.024797622779694,
 0.9720439596873715,
 0.9895789249818887,
 1.0388407806030744,
 1.010613787249961,
 1.3095464037062752,
 1.0733792395181394,
 1.1775161552299303,
 1.0009930557471491,
 1.040052166524046,
 1.0128821808647017,
 0.9735585222337785,
 1.0174996561603091,
 1.0724148654131116,
 1.025461131253809,
 1.0489017864105945,
 1.0042022346854846,
 1.1233701276826904,
 1.

Inserting a new attribute 'Outlier' to the dataframe to include the outlier score of each point

In [15]:
X['Outlier'] = outlier_score

In [16]:
X

Index       State/UT  Year  C1  C2  C3  Male  Female   Outlier
0         0  A & N ISLANDS  2001   0   0   1    27      19  1.114866
1         0  A & N ISLANDS  2001   0   1   0    14       9  1.005314
2         0  A & N ISLANDS  2001   1   0   0    38      22  1.044285
3         1  A & N ISLANDS  2002   0   0   1    33      16  1.042596
4         1  A & N ISLANDS  2002   0   1   0    18      18  1.318613
...     ...            ...   ...  ..  ..  ..   ...     ...       ...
1255    418    WEST BENGAL  2011   0   1   0  3625    3259  1.138203
1256    418    WEST BENGAL  2011   1   0   0  4856    2833  1.146891
1257    419    WEST BENGAL  2012   0   0   1     0       0  1.000000
1258    419    WEST BENGAL  2012   0   1   0     0       0  1.000000
1259    419    WEST BENGAL  2012   1   0   0     0       0  1.000000

[1260 rows x 9 columns]

A scatter plot with x-axis representing number of males and y-axis representing number of females 

In [17]:
fig = go.Figure(data=go.Scatter(x=X['Male'],
                                y=X['Female'],
                                mode='markers',
                                marker_color=X['Male'],
                                text=X['Outlier']))
#fig = px.scatter(X, x="Male", y="Female", text="Outlier", log_x=True, size_max=60)
layout = go.Layout(xaxis=go.XAxis(title='Male'),
                yaxis=go.YAxis(title='Female'),
                )
fig.show()

C:\Users\aniru\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:550: DeprecationWarning:

plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis


C:\Users\aniru\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:578: DeprecationWarning:

plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis




Selecting points with outlier score greater than threshold value(1.75)

In [18]:
y = X[X['Outlier'] > 1.75]

In [19]:
y

Index           State/UT  Year  C1  C2  C3  Male  Female   Outlier
82       27  ARUNACHAL PRADESH  2004   0   1   0     0       4  2.281180
169      56              BIHAR  2009   0   1   0   143     250  2.128948
211      70         CHANDIGARH  2011   0   1   0     5      18  1.943653
271      90       D & N HAVELI  2007   0   1   0    17      25  1.822839
274      91       D & N HAVELI  2008   0   1   0    10      20  1.791194
508     169    JAMMU & KASHMIR  2002   0   1   0    19      38  1.855554
514     171    JAMMU & KASHMIR  2004   0   1   0     6      20  2.011580
688     229     MADHYA PRADESH  2002   0   1   0  1406    1727  1.776889
691     230     MADHYA PRADESH  2003   0   1   0  1462    1745  1.841240
706     235     MADHYA PRADESH  2008   0   1   0  1703    1780  1.841226
709     236     MADHYA PRADESH  2009   0   1   0  1551    1744  1.850450
712     237     MADHYA PRADESH  2010   0   1   0  1569    1955  2.049320
715     238     MADHYA PRADESH  2011   0   1   0  1480    1875  1.986413
787     262            MANIPUR  2011   0   1   0     0       9  2.390989
974     324             PUNJAB  2001   1   0   0    50     114  2.099389
1054    351             SIKKIM  2004   0   1   0     7      23  2.220931
1168    389      UTTAR PRADESH  2006   0   1   0   590     736  1.792551
1240    413        WEST BENGAL  2006   0   1   0  5793    4833  2.094244

In [20]:
a = pd.read_excel('~/DMFinalTerm_TeamID_05/data/data_cleaning.xlsx')

Selecting points with 'STATE/UT' as Madhya Pradesh with causes as Economic/Social problems 

In [21]:
b = a[(a['STATE/UT'] == 'MADHYA PRADESH') & (a['CAUSE'].isin(['Bankruptcy or Sudden change in Economic Status','Suspected/Illicit Relation','Cancellation/Non-Settlement of Marriage','Death of Dear Person','Dowry Dispute','Divorce','Fall in Social Reputation','Family Problems','Love Affairs','Poverty','Professional/Career Problem','Property Dispute','Unemployment']))]

In [22]:
c = b.drop(['Unnamed: 0','STATE/UT','Year','CAUSE','Total'],axis = 1)
c

Male  Female
9048    62      27
9049    45      87
9050    11      26
9057    30      47
9058     0     470
...    ...     ...
9352   187     168
9354    17       3
9355   107       1
9356    69      14
9357    73      41

[156 rows x 2 columns]

In [23]:
cols = c.columns[1:]

Converting the dataset to a numpy array

In [24]:
x=c['Male']
y=c['Female']
d=c[['Male','Female']]
ce=np.array([[d.iloc[0][0],d.iloc[0][1]]])
for i in range(1,len(d)):
    ce=np.append(ce,[[d.iloc[i][0],d.iloc[i][1]]],axis=0)

ce

array([[  62,   27],
       [  45,   87],
       [  11,   26],
       [  30,   47],
       [   0,  470],
       [   1,    2],
       [  62,   35],
       [ 813,  689],
       [  66,   57],
       [  28,   14],
       [  42,   20],
       [  65,   52],
       [  51,   33],
       [  64,   20],
       [  46,   92],
       [  20,   28],
       [  17,   31],
       [   2,  536],
       [   2,    5],
       [  53,   35],
       [ 849,  769],
       [  60,   90],
       [  41,   22],
       [  78,   14],
       [ 109,   52],
       [  65,   33],
       [  70,   15],
       [  65,   78],
       [  18,   13],
       [  31,   31],
       [   0,  516],
       [   0,    7],
       [  71,   63],
       [ 809,  821],
       [  80,   77],
       [  22,   10],
       [  79,   16],
       [  97,   51],
       [ 120,   47],
       [  54,   10],
       [  55,   81],
       [  27,   19],
       [  14,   23],
       [   1,  621],
       [   1,    5],
       [  41,   30],
       [ 686,  585],
       [  84,

In [25]:
ckdist_val = {} # stores the value of k distance of all points
for y in ce:
    ckdist_val[tuple(y)] = kdistance(15,y,ce)
ckdist_val 

{(62, 27): 15.620499351813308,
 (45, 87): 36.61966684720111,
 (11, 26): 18.681541692269406,
 (30, 47): 20.518284528683193,
 (0, 470): 380.7886552931954,
 (1, 2): 19.235384061671343,
 (62, 35): 17.0,
 (813, 689): 814.1406512390841,
 (66, 57): 21.37755832643195,
 (28, 14): 14.035668847618199,
 (42, 20): 15.556349186104045,
 (65, 52): 20.808652046684813,
 (51, 33): 15.811388300841896,
 (64, 20): 15.231546211727817,
 (46, 92): 40.162171256046406,
 (20, 28): 16.492422502470642,
 (17, 31): 19.209372712298546,
 (2, 536): 446.1748536168304,
 (2, 5): 18.0,
 (53, 35): 17.69180601295413,
 (849, 769): 859.3165889240123,
 (60, 90): 35.6931365951495,
 (41, 22): 14.866068747318506,
 (78, 14): 20.615528128088304,
 (109, 52): 40.496913462633174,
 (65, 33): 18.681541692269406,
 (70, 15): 17.204650534085253,
 (65, 78): 26.0,
 (18, 13): 10.816653826391969,
 (31, 31): 15.556349186104045,
 (0, 516): 426.487983418056,
 (0, 7): 19.1049731745428,
 (71, 63): 24.08318915758459,
 (809, 821): 830.2680290123184,
 (

In [26]:
cnn = {} # stores the k nearest neighbours of all points
for y in ce:
    cnn[tuple(y)] = nearestNeighbours(15,y,ce)
cnn[tuple(ce[0])]

array([[65, 33],
       [64, 20],
       [54, 27],
       [62, 35],
       [70, 24],
       [70, 23],
       [69, 18],
       [51, 24],
       [53, 35],
       [51, 33],
       [71, 38],
       [56, 40],
       [70, 15],
       [69, 14],
       [69, 14]], dtype=int64)

In [27]:
coutlier_score = [] # stores the outlier score(lof value) of all points
for y in ce:
    coutlier_score.append(lof(y,15,ce,cnn,ckdist_val))

In [28]:
coutlier_score

[0.9757292129099108,
 1.2060565958376175,
 1.0941925387760096,
 1.0695120703306524,
 2.5425762808023182,
 1.1674518733193955,
 0.9881222088895424,
 1.173232346436831,
 0.9969397610754455,
 0.9823317417345017,
 1.0144557934901757,
 0.9982993415412833,
 0.9758919337827189,
 0.9660824034876495,
 1.278419263715685,
 1.0897249208265067,
 1.1354229567737855,
 2.6064694974756786,
 1.1174880272607646,
 0.971876406058972,
 1.1834370079497372,
 1.1835981112073164,
 0.9924384475263235,
 1.0043204485196993,
 1.3117214853817376,
 0.9841503331285152,
 0.9943003764388209,
 1.0794985496967073,
 0.9709693440842101,
 1.0012931591314236,
 2.5874325084277534,
 1.1608866677799976,
 1.0254105614213918,
 1.172364492302901,
 1.1023531992720943,
 0.9849379343300799,
 1.018959990103218,
 1.1753168800518905,
 1.4240981621216227,
 1.003202784486248,
 1.1356527872146802,
 0.9807968031990247,
 1.051475602678481,
 1.804701410078383,
 1.1442401425826856,
 0.9615593513175504,
 1.3995778159088583,
 1.11108183416118,
 0

Appending the attribute 'Outlier' representing the outlier score to the dataframe

In [29]:
b['Outlier'] = coutlier_score

C:\Users\aniru\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Scatter plot with x-axis representing number of males and y-axis representing number of females

In [30]:
fig = go.Figure(data=go.Scatter(x=b['Male'],
                                y=b['Female'],
                                mode='markers',
                                marker_color=b['Male'],
                                text=b['Outlier']))

fig.show()

Selecting the points with outlier score greater than the threshold value(1.75)

In [31]:
b[b['Outlier'] > 1.75]

Unnamed: 0        STATE/UT  Year                        CAUSE  Male  \
9058        9994  MADHYA PRADESH  2001                Dowry Dispute     0   
9084       10022  MADHYA PRADESH  2002                Dowry Dispute     2   
9110       10050  MADHYA PRADESH  2003                Dowry Dispute     0   
9136       10078  MADHYA PRADESH  2004                Dowry Dispute     1   
9162       10106  MADHYA PRADESH  2005                Dowry Dispute     0   
9188       10134  MADHYA PRADESH  2006                Dowry Dispute     0   
9193       10139  MADHYA PRADESH  2006              Family Problems   730   
9214       10162  MADHYA PRADESH  2007                Dowry Dispute     0   
9219       10167  MADHYA PRADESH  2007              Family Problems   693   
9240       10190  MADHYA PRADESH  2008                Dowry Dispute    10   
9241       10191  MADHYA PRADESH  2008                      Divorce     1   
9251       10201  MADHYA PRADESH  2008  Professional/Career Problem   166   
9266       10218  MADHYA PRADESH  2009                Dowry Dispute     0   
9277       10229  MADHYA PRADESH  2009  Professional/Career Problem   129   
9292       10246  MADHYA PRADESH  2010                Dowry Dispute     7   
9296       10250  MADHYA PRADESH  2010    Fall in Social Reputation   153   
9300       10254  MADHYA PRADESH  2010                 Love Affairs   115   
9318       10274  MADHYA PRADESH  2011                Dowry Dispute     8   
9326       10282  MADHYA PRADESH  2011                 Love Affairs   153   
9344       10302  MADHYA PRADESH  2012                Dowry Dispute     1   
9352       10310  MADHYA PRADESH  2012                 Love Affairs   187   

      Female   Total   Outlier  
9058     470   470.0  2.542576  
9084     536   538.0  2.606469  
9110     516   516.0  2.587433  
9136     621   622.0  1.804701  
9162     517   517.0  2.588418  
9188     585   585.0  2.656256  
9193     564  1294.0  1.816311  
9214     588   588.0  2.659281  
9219     528  1221.0  1.792104  
9240     748   758.0  1.868000  
9241      51    52.0  1.750772  
9251       3   169.0  2.521709  
9266     666   666.0  1.827808  
9277       5   134.0  1.752191  
9292     769   776.0  1.880960  
9296      93   246.0  2.164253  
9300     139   254.0  2.318749  
9318     693   701.0  1.838683  
9326     145   298.0  2.781291  
9344     630   631.0  1.809146  
9352     168   355.0  3.406789